In [3]:
import pandas as pd

data = pd.read_csv('BTCUSDT_1h_data.csv')
data

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,1663599600000,19303.37,19346.59,19051.00,19080.49,20697.73875,1663603199999,3.976865e+08,311689,10337.84457,1.986580e+08,0
1,1663603200000,19081.24,19231.27,19063.55,19207.61,11617.87392,1663606799999,2.226842e+08,201387,5922.85489,1.135268e+08,0
2,1663606800000,19207.61,19219.17,18926.83,18964.59,13102.03544,1663610399999,2.495837e+08,225015,6228.83020,1.186340e+08,0
3,1663610400000,18963.78,19166.13,18877.00,19159.54,12496.92584,1663613999999,2.374773e+08,212567,6478.89791,1.231412e+08,0
4,1663614000000,19160.39,19560.35,19153.68,19514.80,23381.35401,1663617599999,4.532573e+08,360030,12408.07016,2.405323e+08,0
...,...,...,...,...,...,...,...,...,...,...,...,...
17514,1726653600000,59654.76,59924.00,59348.88,59907.49,2111.03734,1726657199999,1.258120e+08,199698,1283.06987,7.646234e+07,0
17515,1726657200000,59907.50,60000.00,59821.81,59846.00,451.89241,1726660799999,2.707379e+07,80886,222.44433,1.332743e+07,0
17516,1726660800000,59846.00,60000.00,59689.57,59978.24,829.14742,1726664399999,4.962566e+07,122768,456.52567,2.733147e+07,0
17517,1726664400000,59978.24,60160.00,59308.61,59508.50,1920.77662,1726667999999,1.147251e+08,265786,972.55835,5.809038e+07,0


In [14]:
pd.to_datetime(data['Open time'], unit='ms')

0       2022-09-19 15:00:00
1       2022-09-19 16:00:00
2       2022-09-19 17:00:00
3       2022-09-19 18:00:00
4       2022-09-19 19:00:00
                ...        
17514   2024-09-18 10:00:00
17515   2024-09-18 11:00:00
17516   2024-09-18 12:00:00
17517   2024-09-18 13:00:00
17518   2024-09-18 14:00:00
Name: Open time, Length: 17519, dtype: datetime64[ns]

In [1]:
data_path = './my_data'

In [7]:
import os

for file in os.listdir(data_path):
    print(file[:-9])

ETHUSDT_1h
ETHUSDT_15m
BNBUSDT_15m
BTCUSDT_15m
BNBUSDT_1h
BTCUSDT_1h


In [11]:
data_path = './my_data/'
new_data_path = './my_data_new/'
import pandas as pd

for file in os.listdir(data_path):
    data = pd.read_csv(data_path+file)
    data['Open time'] = pd.to_datetime(data['Open time'], unit='ms')
    data['Close time'] = pd.to_datetime(data['Close time'], unit='ms')
    data['code'] = file[:-9]
    data.to_csv(new_data_path+file, index=None)

In [2]:
from data_collection.qlib_dump_bin import DumpDataAll

DumpDataAll(
        csv_path='./my_data_new',
        qlib_dir='./my_data_feature',
        max_workers=50,
        exclude_fields="Open time,code,Close time,Ignore",
        symbol_field_name="code",
        date_field_name='Open time'
).dump()

2024-09-21 21:37:01.685 | INFO     | data_collection.qlib_dump_bin:_get_all_date:272 - start get all date......
100%|██████████| 6/6 [00:01<00:00,  3.23it/s]
2024-09-21 21:37:03.543 | INFO     | data_collection.qlib_dump_bin:_get_all_date:291 - end of get all date.

2024-09-21 21:37:03.544 | INFO     | data_collection.qlib_dump_bin:_dump_calendars:294 - start dump calendars......
2024-09-21 21:37:03.723 | INFO     | data_collection.qlib_dump_bin:_dump_calendars:297 - end of calendars dump.

2024-09-21 21:37:03.723 | INFO     | data_collection.qlib_dump_bin:_dump_instruments:300 - start dump instruments......
2024-09-21 21:37:03.724 | INFO     | data_collection.qlib_dump_bin:_dump_instruments:302 - end of instruments dump.

2024-09-21 21:37:03.724 | INFO     | data_collection.qlib_dump_bin:_dump_features:305 - start dump features......
  0%|          | 0/6 [00:00<?, ?it/s]/Users/andres/Desktop/Kronos Research/alphagen/data_collection/qlib_dump_bin.py:198: FutureWarning: Passing unit-les

In [12]:
import pandas as pd
import json
import os
from typing import Optional, Tuple
from datetime import datetime
import fire

import numpy as np
from sb3_contrib.ppo_mask import MaskablePPO
from stable_baselines3.common.callbacks import BaseCallback
from alphagen.data.calculator import AlphaCalculator

from alphagen.data.expression import *
from alphagen.models.alpha_pool import AlphaPool, AlphaPoolBase
from alphagen.rl.env.wrapper import AlphaEnv
from alphagen.rl.policy import LSTMSharedNet
from alphagen.utils.random import reseed_everything
from alphagen.rl.env.core import AlphaEnvCore
from alphagen_qlib.calculator import QLibStockDataCalculator

from data_collection.qlib_dump_bin import DumpDataAll

data_path = './my_data/original/'
new_data_path = './my_data/basic/'

In [13]:
def convert_qlib():
    for file in os.listdir(data_path):
        data = pd.read_csv(data_path+file)
        data['Open time'] = pd.to_datetime(data['Open time'], unit='ms')
        data['Close time'] = pd.to_datetime(data['Close time'], unit='ms')
        data['code'] = file[:-9]
        data.columns = data.columns.str.replace(' ', '_')
        data.to_csv(new_data_path+file, index=None)

    DumpDataAll(
            csv_path='./my_data/basic',
            qlib_dir='./my_data/qlib',
            max_workers=50,
            exclude_fields="Open_time,code,Close_time,Ignore",
            symbol_field_name="code",
            date_field_name='Open_time',
            freq='min'
    ).dump()

In [14]:
convert_qlib()

2024-09-22 09:57:43.592 | INFO     | data_collection.qlib_dump_bin:_get_all_date:272 - start get all date......
100%|██████████| 6/6 [00:01<00:00,  3.13it/s]
2024-09-22 09:57:45.512 | INFO     | data_collection.qlib_dump_bin:_get_all_date:291 - end of get all date.

2024-09-22 09:57:45.514 | INFO     | data_collection.qlib_dump_bin:_dump_calendars:294 - start dump calendars......
2024-09-22 09:57:45.705 | INFO     | data_collection.qlib_dump_bin:_dump_calendars:297 - end of calendars dump.

2024-09-22 09:57:45.705 | INFO     | data_collection.qlib_dump_bin:_dump_instruments:300 - start dump instruments......
2024-09-22 09:57:45.706 | INFO     | data_collection.qlib_dump_bin:_dump_instruments:302 - end of instruments dump.

2024-09-22 09:57:45.707 | INFO     | data_collection.qlib_dump_bin:_dump_features:305 - start dump features......
  0%|          | 0/6 [00:00<?, ?it/s]/Users/andres/Desktop/Kronos Research/alphagen/data_collection/qlib_dump_bin.py:198: FutureWarning: Passing unit-les

In [1]:
from alphagen_qlib.stock_data import StockData
import qlib
# from qlib.config import REG_CN
from qlib.config import REG_US
# qlib.init(provider_uri="~/.qlib/qlib_data/cn_data_rolling", region=REG_CN)
qlib.init(provider_uri="./my_data/qlib", region=REG_US)

data_train = StockData(instrument='all',
                        start_time='2023-06-20 00:00:00',
                        end_time='2023-07-19 00:00:00')

[46045:MainThread](2024-09-22 10:41:58,593) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[46045:MainThread](2024-09-22 10:41:58,731) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[46045:MainThread](2024-09-22 10:41:58,732) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/andres/Desktop/Kronos Research/alphagen/my_data/qlib')}
[46045:MainThread](2024-09-22 10:41:58,733) INFO - qlib.Initialization - [config.py:416] - default_conf: client.
[46045:MainThread](2024-09-22 10:41:58,734) INFO - qlib.Initialization - [__init__.py:74] - qlib successfully initialized based on client settings.
[46045:MainThread](2024-09-22 10:41:58,734) INFO - qlib.Initialization - [__init__.py:76] - data_path={'__DEFAULT_FREQ': PosixPath('/Users/andres/Desktop/Kronos Research/alphagen/my_data/qlib')}


['$open', '$close', '$high', '$low', '$volume', '$taker_buy_quote_asset_volume', '$taker_buy_base_asset_volume', '$quote_asset_volume', '$number_of_trades']


In [2]:
data_train.data.shape

torch.Size([2915, 9, 6])